# KVecinos en un dataset de detección de spam

Se propone emplear un clasificador basado en distancias sobre el dataset id=44 de openml de detección de Spam. Son un total de 4601 muestras con 57 características.

In [2]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from warnings import filterwarnings

## Descarga del dataset Spam
filterwarnings('ignore')
X, y = fetch_openml(data_id=44, as_frame=False, cache=True, return_X_y=True)
print(X.shape)

## Partición train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)


(4601, 57)


## El clasificador por los vecinos más cercanos

In [4]:
from sklearn.neighbors import KNeighborsClassifier

kv = KNeighborsClassifier()
acc=kv.fit(X_train,y_train).score(X_test,y_test)

print(f'Precisión: {acc:.1%}')

Precisión: 79.5%


**Ejercicio:** Explora el principal parámetros del KNN (n_neighbors) y realiza una búsqueda mediante alguna técnica de optimización ya vista en la práctica anterior

In [5]:
import warnings; warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split, GridSearchCV
G = {'n_neighbors': [1,2,5,10], 'weights':['uniform', 'distance'], 'leaf_size': [5,10,20,30,50], 'p': [1,2,3]}
GS = GridSearchCV(kv, G, scoring='accuracy', refit=True, cv=5)
acc = GS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {GS.best_params_}')

Precisión: 86.4% con {'leaf_size': 5, 'n_neighbors': 10, 'p': 1, 'weights': 'distance'}


## Mejoras

La función de distancia empleada por defecto es la distancia euclídea. Dicha distancia requiere un preproceso de las muestras para que tengan una escala similar todas ellas. Además KNN podría beneficiarse de una proyección mediante PCA con el fin de reducir la dimensionalidad.

**Ejercicio:** Implementa un pipeline con la normalización de los datos y un PCA, seguido del KNN. Busca los mejores parámetros. Se podría conseguir una tasa de acierto >90%.


In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

scaler = StandardScaler()
pca = PCA()
knn = KNeighborsClassifier()

pipe = Pipeline(steps=[("scaler", scaler), ("pca", pca), ("knn", knn)])

G = {"pca__n_components": [5,10,20,50,100], "knn__n_neighbors": [1,3,5,10]}
GS = GridSearchCV(pipe, G, scoring='accuracy', refit=True, cv=5)
acc = GS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {GS.best_params_}')

Precisión: 91.0% con {'knn__n_neighbors': 5, 'pca__n_components': 20}


También podríamos probar diferentes funciones de distancia [sklearn distances](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.distance_metrics.html#sklearn.metrics.pairwise.distance_metrics) a emplear en el parámetro "metric". Así mismo podríamos explorar el parámetro "weights" que pondera el voto de cada vecino de forma diferente según el parámetro escogido.

**Ejercicio:** prueba también diferentes métricas y "weights" junto con todo lo anterior. Emplea el BayessianOpt visto en la práctica anterior.

In [7]:
!pip install scikit-optimize


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
# Solución
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

# Probar solo 10 combinaciones de parámetros, n_iter=10
scaler = StandardScaler()
pca = PCA()
knn = KNeighborsClassifier()

pipe = Pipeline(steps=[("scaler", scaler), ("pca", pca), ("knn", knn)])

G = {"pca__n_components": Integer(1,57),
     "knn__n_neighbors": Integer(1,20),
     "knn__metric": Categorical(["l1","l2"]),
     "knn__weights":Categorical(["uniform","distance"])}

BS = BayesSearchCV(pipe, G, scoring='accuracy', n_iter=20, refit=True, cv=5)

acc = BS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {BS.best_params_}')

Precisión: 92.1% con OrderedDict([('knn__metric', 'l2'), ('knn__n_neighbors', 11), ('knn__weights', 'distance'), ('pca__n_components', 29)])


In [14]:
# Solución
from skopt import BayesSearchCV
from sklearn.preprocessing import PolynomialFeatures
from skopt.space import Real, Categorical, Integer

# Probar solo 10 combinaciones de parámetros, n_iter=10
scaler = StandardScaler()
pca = PCA()
poly = PolynomialFeatures()
knn = KNeighborsClassifier()

pipe = Pipeline(steps=[("scaler", scaler), ("pca", pca), ("poly", poly), ("knn", knn)])

G = {"pca__n_components": Integer(1,57),
     "poly__degree": [2], 
     "knn__n_neighbors": Integer(1,20),
     "knn__metric": Categorical(["l1","l2"]),
     "knn__weights": Categorical(["uniform","distance"]),
     'knn__leaf_size': Integer(1,50),
     'knn__p': Categorical([1,2,3])}

BS = BayesSearchCV(pipe, G, scoring='accuracy', n_iter=20, refit=True, cv=5)

acc = BS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {BS.best_params_}')

Precisión: 90.9% con OrderedDict([('knn__leaf_size', 50), ('knn__metric', 'l2'), ('knn__n_neighbors', 8), ('knn__p', 1), ('knn__weights', 'distance'), ('pca__n_components', 57), ('poly__degree', 2)])


## Olivetti Faces

Prueba ahora el clasificador KNN junto con todos los parámetros y preprocesos que creas convenientes sobre el dataset de reconocimiento facial de Olivetti.

In [2]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_olivetti_faces

## Descarga del dataset Spam
X, y = fetch_olivetti_faces(return_X_y=True)
print(X.shape)

## Partición train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)

(400, 4096)


In [5]:
import warnings; warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split, GridSearchCV
G = {'n_neighbors': [1,2,5,10], 'weights':['uniform', 'distance'], 'leaf_size': [5,10,20,30,50], 'p': [1,2,3]}
GS = GridSearchCV(kv, G, scoring='accuracy', refit=True, cv=5)
acc = GS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {GS.best_params_}')

Precisión: 96.2% con {'leaf_size': 5, 'n_neighbors': 1, 'p': 1, 'weights': 'uniform'}


In [15]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_olivetti_faces
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

X, y = fetch_olivetti_faces(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)

scaler = StandardScaler()
pca = PCA()
poly = PolynomialFeatures()
knn = KNeighborsClassifier()

pipe = Pipeline(steps=[("scaler", scaler), ("pca", pca), ("poly", poly), ("knn", knn)])

G = {"pca__n_components": Integer(1,57),
     "poly__degree": [2], 
     "knn__n_neighbors": Integer(1,20),
     "knn__metric": Categorical(["l1","l2"]),
     "knn__weights": Categorical(["uniform","distance"]),
     'knn__leaf_size': Integer(1,50),
     'knn__p': Categorical([1,2,3])}

BS = BayesSearchCV(pipe, G, scoring='accuracy', n_iter=20, refit=True, cv=5)

acc = BS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {BS.best_params_}')

Precisión: 96.2% con OrderedDict([('knn__leaf_size', 44), ('knn__metric', 'l2'), ('knn__n_neighbors', 1), ('knn__p', 1), ('knn__weights', 'distance'), ('pca__n_components', 31), ('poly__degree', 2)])


In [20]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_olivetti_faces
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

X, y = fetch_olivetti_faces(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)

scaler = StandardScaler()
pca = PCA()
poly = PolynomialFeatures()
knn = KNeighborsClassifier()

pipe = Pipeline(steps=[("scaler", scaler), ("pca", pca), ("poly", poly), ("knn", knn)])

G = {"pca__n_components": Integer(1,30),
     "poly__degree": Integer(1,3), 
     "knn__n_neighbors": Integer(1,20),
     "knn__metric": Categorical(["l1","l2"]),
     "knn__weights": Categorical(["uniform","distance"]),
     "knn__algorithm": Categorical(["auto", "ball_tree", "kd_tree", "brute"]),
     "knn__leaf_size": Integer(1,50),
     "knn__p": Real(1,10)}

BS = BayesSearchCV(pipe, G, scoring='accuracy', n_iter=20, refit=True, cv=5)

acc = BS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {BS.best_params_}')

Precisión: 95.0% con OrderedDict([('knn__algorithm', 'kd_tree'), ('knn__leaf_size', 37), ('knn__metric', 'l1'), ('knn__n_neighbors', 1), ('knn__p', 1.0), ('knn__weights', 'distance'), ('pca__n_components', 19), ('poly__degree', 1)])
